In [3]:
import numpy as np
try:
    from docplex.mp.model import Model
except:
    !pip install docplex
from docplex.mp.model import Model

import pandas as pd

try:
    import ipywidgets
except:
    !pip install ipywidgets
from ipywidgets import interact
import ipywidgets as widgets
import time

In [1]:
# Number of nurses
N = 15 #arbitrary data
nurses = ['Enfermera_' +str(n) for n in range(N)]
# period of days that we want to schedule
T = 7 #one week
days = ['Dia_' +str(t) for t in range(T)]
# Shifts assuming 3 types of shifts
S = ['Mañana', 'Tarde', 'Noche']
# number of nurses required per shift
R = {'Mañana' : 5,
     'Tarde' : 4,
     'Noche' : 3}
# Weights assigned to each shift
W = {'Mañana' : 10,
     'Tarde' : 10,
     'Noche' : 10}

In [ ]:
# Temporary variables that could fix the work of certain data
days2 = ['Dia_' +str(t+1) for t in range(T-1)]
days3 = ['Dia_' +str(t) for t in range(6)]

In [ ]:
mdl = Model('Scheduling')
# Create variables
idx_x = [(i,s,t) for i in nurses for s in S for t in days]
idd=[(s) for s in S]
x = mdl.binary_var_dict(idx_x) #Dictionary with days, personnel name and corresponding shift
D_dif = mdl.continuous_var #Difference between the maximum and minimum of day shifts worked
E_dif = mdl.continuous_var #Difference between the maximum and minimum of evening shifts worked
N_dif = mdl.continuous_var #Difference between the maximum and minimum night shifts worked
O_dif = mdl.continuous_var #Difference between the maximum and minimum of days off
mini = mdl.continuous_var_dict(idd) #Minimum number of shifts assigned to nurses in the calendar
maxi = mdl.continuous_var_dict(idd) #Maximum number of shifts assigned to nurses in the calendar

## Optimization model 

\begin{equation*}
\begin{aligned}
& \underset{x_{ist}\in\{0,1\}}{\text{min}}(\sum _{s \in S }(( \underset{i \in 0,1,\ldots , N}{\max}\sum_{t=1}^T x_{ist}-\underset{i \in 0,1,\ldots , N}{\min}\sum_{t=1}^T x_{ist})\cdot W(s)))\\
\end{aligned}
\end{equation*}

In [4]:
mdl.minimize(mdl.sum(maxi[s]-mini[s]*W[s] for s in S)) 
#mdl.minimize(mdl.sum(x[i,s,t]*W[s] for i in nurses for s in S for t in days))#Intento tipo 1 minimizar diferencia entre maximos y minimos con peso

In [5]:
#El perrsonal no puede tener más de un turno al día
mdl.add_constraints(mdl.sum(x[i,s,t] for s in S) <= 1 for i in nurses for t in days);
#Se cumple con tener el personal asignado para cada turno del día
mdl.add_constraints(mdl.sum(x[i,s,t] for i in nurses)>= R[s]  for s in S for t in days);
#No se trabaja un día si se trabaja una noche el día anterior.
mdl.add_constraints(mdl.sum(x[i,'Mañana',t] for i in nurses for t in days)-1 <= x[i,'Noche',t] for i in nurses for t in days2);
#Cada integrante del personal no trabaja más de una cantidad especifica de noches
mdl.add_constraints( mdl.sum(x[i,'Noche',t] for i in nurses)<= 6 for t in days ); 

In [7]:
mdl.add_kpi(mdl.max(mdl.sum(x[i,s,t]*h[s] for s in S for t in days)for i in nurses), 'Maximum # hours worked')
mdl.add_kpi(mdl.min(mdl.sum(x[i,s,t]*h[s] for s in S for t in days)for i in nurses), 'Minimum # hours worked');

### Solve the problem

In [8]:
mdl.print_information()
mdl.solve()
mdl.solution.solve_details

Model: Scheduling
 - number of variables: 345
   - binary=315, integer=0, continuous=30
 - number of constraints: 276
   - linear=276
 - parameters: defaults
 - objective: minimize
 - problem type is: MILP


docplex.mp.SolveDetails(time=0.063,status='integer optimal solution')

In [9]:
mdl.report()

* model Scheduling solved with objective = 658.000
*  KPI: Maximum # hours worked = 55.000
*  KPI: Minimum # hours worked = 36.000


In [10]:
status = mdl.solve_details.status == 'integer optimal solution'
status

True

## Analysing the solution

Below we provide simple tools to analyse the solution obtained

In [11]:
def x_star_to_pandas(x):
    '''
    takes in input the solution of the optimization problem as a dictionary 
    returns the solution as a dataframe 
    '''
    sol = pd.DataFrame(columns = ['Nurse', 'Shift', 'Day'])
    k = 0
    for key, value in x.items():
        if value>0:
            sol.loc[k] =np.array([i for i in key])
            k+=1
    return sol

In [17]:
# transform the solution into a dataframe
x_star_dict =mdl.solution.get_value_dict(x)
sol_x = x_star_to_pandas(x_star_dict)
sol_x.head()

,Nurse,Shift,Day
0,Nurse_0,Morning,Day_1
1,Nurse_0,Morning,Day_3
2,Nurse_0,Afternoon,Day_2
3,Nurse_0,Afternoon,Day_6
4,Nurse_0,Night,Day_4


### How many hours does each nurse work over the period?

In [14]:
worked_hours = {n:0 for n in nurses}

for i,j in sol_x.iterrows():
    worked_hours[j['Nurse']]+=h[j['Shift']]
worked_hours

{'Nurse_0': 39,
 'Nurse_1': 37,
 'Nurse_2': 52,
 'Nurse_3': 53,
 'Nurse_4': 39,
 'Nurse_5': 46,
 'Nurse_6': 40,
 'Nurse_7': 47,
 'Nurse_8': 40,
 'Nurse_9': 55,
 'Nurse_10': 42,
 'Nurse_11': 53,
 'Nurse_12': 40,
 'Nurse_13': 39,
 'Nurse_14': 36}

### Average of hours worked by day

In [15]:
for i, j in worked_hours.items():
    print(i,':',j/T)

Nurse_0 : 5.571428571428571
Nurse_1 : 5.285714285714286
Nurse_2 : 7.428571428571429
Nurse_3 : 7.571428571428571
Nurse_4 : 5.571428571428571
Nurse_5 : 6.571428571428571
Nurse_6 : 5.714285714285714
Nurse_7 : 6.714285714285714
Nurse_8 : 5.714285714285714
Nurse_9 : 7.857142857142857
Nurse_10 : 6.0
Nurse_11 : 7.571428571428571
Nurse_12 : 5.714285714285714
Nurse_13 : 5.571428571428571
Nurse_14 : 5.142857142857143


### Visualization tool

Below we provide a tool to check the schedule. 

In [16]:
# remove warning from pandas (in the viz_tool it does what we need)
import warnings
warnings.simplefilter(action='ignore')

In [18]:

def viz_tool(nurse,shift,day):
    '''
    interactive function to extract the information required:
    if a value is 'All' then it returns all the values for that specific feature
    '''
    global nurses,S,days
    
    if nurse == 'All':
        df_tmp = sol_x[(sol_x['Nurse'].isin(nurses))]
    else:
        df_tmp = sol_x[(sol_x['Nurse']==nurse)]

    if shift == 'All':
        df_tmp = df_tmp[(sol_x['Shift'].isin(S))]
    else:
        df_tmp = df_tmp[(sol_x['Shift']==shift)]

    if day == 'All':
        df_tmp = df_tmp[(sol_x['Day'].isin(days))]    
    else:
        df_tmp = df_tmp[(sol_x['Day']==day)]

    print(df_tmp)

interact(viz_tool, nurse = widgets.Dropdown(value="All",placeholder='Type something', options=nurses+['All']),
              shift=widgets.Dropdown(value='All',placeholder='Type something', options=S+['All']),
              day = widgets.Dropdown(value="All",placeholder='Type something', options=days+['All'])
        );



interactive(children=(Dropdown(description='nurse', index=15, options=('Nurse_0', 'Nurse_1', 'Nurse_2', 'Nurse…